In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('season_df_v1.csv')

df.head()

,game_id,ball_possession_home,ball_possession_away,expected_goals_home,expected_goals_away,big_chances_home,big_chances_away,total_shots_home,total_shots_away,goalkeeper_saves_home,...,big_chances_missed_home,big_chances_missed_away,punches_home,punches_away,red_cards_home,red_cards_away,errors_lead_to_a_goal_home,errors_lead_to_a_goal_away,penalty_saves_home,penalty_saves_away
0,12436410,63.0,37.0,0.10,1.05,0.0,1.0,1.0,5.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12436419,41.0,59.0,0.97,0.37,2.0,0.0,9.0,6.0,0.0,...,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,12436434,58.0,42.0,0.66,0.53,2.0,2.0,5.0,2.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,12436436,51.0,49.0,2.30,1.65,4.0,4.0,14.0,13.0,4.0,...,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12436437,68.0,32.0,0.27,1.56,0.0,3.0,5.0,6.0,0.0,...,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = df.rename(columns={"game_id": "match_id"})
df.to_csv('season_df_v1.csv', index=False)

In [9]:
import json

with open('matches_data.json', 'r', encoding='utf-8') as f:
    matches = json.load(f)

In [11]:
match_lookup = {}
for match in matches:
    match_id = match['id']
    match_lookup[match_id] = {
        'league_id': match['tournament']['uniqueTournament']['id'],
        'season_id': match['season']['id'],
        'home_team_id': match['homeTeam']['id'],
        'away_team_id': match['awayTeam']['id']
    }

df['league_id'] = df['match_id'].map(lambda x: match_lookup.get(x, {}).get('league_id'))
df['season_id'] = df['match_id'].map(lambda x: match_lookup.get(x, {}).get('season_id'))
df['home_team_id'] = df['match_id'].map(lambda x: match_lookup.get(x, {}).get('home_team_id'))
df['away_team_id'] = df['match_id'].map(lambda x: match_lookup.get(x, {}).get('away_team_id'))


In [15]:
df.to_csv('season_df_v1.csv', index=False)

In [30]:
momentum_df = pd.read_csv('pc_momentum.csv')
momentum_df

,match_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,84.0,85.0,86.0,87.0,88.0,89.0,90.0,momentum_pc1,PC1_score,PC1_sigmoid
0,12436410,2,-3,-11,-28,-14,-16,-18,-42,-58,...,5,7,6,3,6,31,22,-3.941367,-1.044258,-9.618967
1,12436419,-6,-9,-40,-35,-31,-66,-49,-28,-23,...,5,-19,-14,-13,-7,-6,-1,-2.913133,-0.771829,-8.969837
2,12436434,-2,2,-7,-9,-44,-98,-60,-53,-33,...,17,43,47,22,-33,-29,-16,-1.436972,-0.380723,-6.159707
3,12436436,3,6,-5,-41,-6,5,35,-48,-21,...,-28,-13,-40,-38,-30,-46,-44,-1.882041,-0.498643,-7.356908
4,12436437,-3,-5,3,4,7,9,-36,-17,-19,...,29,50,56,47,32,38,32,1.753782,0.464661,7.048585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,13344418,-5,-15,-5,29,53,25,1,-6,-23,...,-23,-13,22,5,2,-28,-36,-1.692906,-0.448532,-6.892119
375,13365051,-7,-65,-6,11,15,20,27,57,37,...,-20,-13,-4,-3,-19,-13,-10,0.610463,0.161741,2.960929
376,13560219,2,66,18,21,18,19,15,19,25,...,17,42,27,19,15,11,2,1.100644,0.291614,5.007614
377,13560222,-3,15,-9,6,10,22,18,21,15,...,-62,-44,-68,-54,-54,-64,-49,-3.968049,-1.051327,-9.628814


In [31]:
import numpy as np

momentum_df['home_momentum'] = np.where(momentum_df['PC1_sigmoid'] >= 0, momentum_df['PC1_sigmoid'], 10 - momentum_df['PC1_sigmoid'].abs())
momentum_df['away_momentum'] = np.where(momentum_df['PC1_sigmoid'] < 0, momentum_df['PC1_sigmoid'].abs(), 10 - momentum_df['PC1_sigmoid'].abs())

momentum_df

,match_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,86.0,87.0,88.0,89.0,90.0,momentum_pc1,PC1_score,PC1_sigmoid,home_momentum,away_momentum
0,12436410,2,-3,-11,-28,-14,-16,-18,-42,-58,...,6,3,6,31,22,-3.941367,-1.044258,-9.618967,0.381033,9.618967
1,12436419,-6,-9,-40,-35,-31,-66,-49,-28,-23,...,-14,-13,-7,-6,-1,-2.913133,-0.771829,-8.969837,1.030163,8.969837
2,12436434,-2,2,-7,-9,-44,-98,-60,-53,-33,...,47,22,-33,-29,-16,-1.436972,-0.380723,-6.159707,3.840293,6.159707
3,12436436,3,6,-5,-41,-6,5,35,-48,-21,...,-40,-38,-30,-46,-44,-1.882041,-0.498643,-7.356908,2.643092,7.356908
4,12436437,-3,-5,3,4,7,9,-36,-17,-19,...,56,47,32,38,32,1.753782,0.464661,7.048585,7.048585,2.951415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,13344418,-5,-15,-5,29,53,25,1,-6,-23,...,22,5,2,-28,-36,-1.692906,-0.448532,-6.892119,3.107881,6.892119
375,13365051,-7,-65,-6,11,15,20,27,57,37,...,-4,-3,-19,-13,-10,0.610463,0.161741,2.960929,2.960929,7.039071
376,13560219,2,66,18,21,18,19,15,19,25,...,27,19,15,11,2,1.100644,0.291614,5.007614,5.007614,4.992386
377,13560222,-3,15,-9,6,10,22,18,21,15,...,-68,-54,-54,-64,-49,-3.968049,-1.051327,-9.628814,0.371186,9.628814


In [32]:
momentum_df['home_momentum'] = momentum_df['home_momentum'].round(2)
momentum_df['away_momentum'] = momentum_df['away_momentum'].round(2)

In [33]:
momentum_df

,match_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,86.0,87.0,88.0,89.0,90.0,momentum_pc1,PC1_score,PC1_sigmoid,home_momentum,away_momentum
0,12436410,2,-3,-11,-28,-14,-16,-18,-42,-58,...,6,3,6,31,22,-3.941367,-1.044258,-9.618967,0.38,9.62
1,12436419,-6,-9,-40,-35,-31,-66,-49,-28,-23,...,-14,-13,-7,-6,-1,-2.913133,-0.771829,-8.969837,1.03,8.97
2,12436434,-2,2,-7,-9,-44,-98,-60,-53,-33,...,47,22,-33,-29,-16,-1.436972,-0.380723,-6.159707,3.84,6.16
3,12436436,3,6,-5,-41,-6,5,35,-48,-21,...,-40,-38,-30,-46,-44,-1.882041,-0.498643,-7.356908,2.64,7.36
4,12436437,-3,-5,3,4,7,9,-36,-17,-19,...,56,47,32,38,32,1.753782,0.464661,7.048585,7.05,2.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,13344418,-5,-15,-5,29,53,25,1,-6,-23,...,22,5,2,-28,-36,-1.692906,-0.448532,-6.892119,3.11,6.89
375,13365051,-7,-65,-6,11,15,20,27,57,37,...,-4,-3,-19,-13,-10,0.610463,0.161741,2.960929,2.96,7.04
376,13560219,2,66,18,21,18,19,15,19,25,...,27,19,15,11,2,1.100644,0.291614,5.007614,5.01,4.99
377,13560222,-3,15,-9,6,10,22,18,21,15,...,-68,-54,-54,-64,-49,-3.968049,-1.051327,-9.628814,0.37,9.63


In [37]:
df = df.merge(momentum_df[['match_id', 'home_momentum', 'away_momentum']], on='match_id', how='left')
df

,match_id,ball_possession_home,ball_possession_away,expected_goals_home,expected_goals_away,big_chances_home,big_chances_away,total_shots_home,total_shots_away,goalkeeper_saves_home,...,errors_lead_to_a_goal_home,errors_lead_to_a_goal_away,penalty_saves_home,penalty_saves_away,league_id,season_id,home_team_id,away_team_id,home_momentum,away_momentum
0,12436410,63.0,37.0,0.10,1.05,0.0,1.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,17,61627,50,42,0.38,9.62
1,12436419,41.0,59.0,0.97,0.37,2.0,0.0,9.0,6.0,0.0,...,0.0,1.0,0.0,0.0,17,61627,7,38,1.03,8.97
2,12436434,58.0,42.0,0.66,0.53,2.0,2.0,5.0,2.0,0.0,...,1.0,0.0,0.0,0.0,17,61627,48,40,3.84,6.16
3,12436436,51.0,49.0,2.30,1.65,4.0,4.0,14.0,13.0,4.0,...,0.0,0.0,0.0,0.0,17,61627,50,17,2.64,7.36
4,12436437,68.0,32.0,0.27,1.56,0.0,3.0,5.0,6.0,0.0,...,0.0,0.0,0.0,0.0,17,61627,3,14,7.05,2.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,13344418,43.0,57.0,0.35,0.39,0.0,1.0,7.0,3.0,1.0,...,0.0,0.0,0.0,0.0,17,61627,48,44,3.11,6.89
376,13365051,50.0,50.0,0.39,0.29,2.0,0.0,3.0,8.0,1.0,...,0.0,0.0,0.0,0.0,17,61627,37,31,2.96,7.04
377,13560219,64.0,36.0,0.33,0.47,0.0,1.0,4.0,6.0,1.0,...,0.0,0.0,0.0,0.0,17,61627,31,39,5.01,4.99
378,13560222,31.0,69.0,0.35,0.21,1.0,0.0,3.0,3.0,3.0,...,0.0,1.0,0.0,0.0,17,61627,60,43,0.37,9.63


In [38]:
df.to_csv('season_df_v1.csv', index=False)

In [43]:
df['xg_diff'] = df['expected_goals_home'] - df['expected_goals_away']
df['xg_diff_home'] = df['expected_goals_home'] - df['expected_goals_away']
df['xg_diff_away'] = df['expected_goals_away'] - df['expected_goals_home']

df[['xg_diff', 'xg_diff_home', 'xg_diff_away']]

df.to_csv('season_df_v1.csv', index=False)

In [44]:
df['total_goals'] = df['home_normaltime'] + df['away_normaltime']
df['total_goals']

0      4
1      2
2      1
3      4
4      3
      ..
375    4
376    2
377    3
378    1
379    4
Name: total_goals, Length: 380, dtype: int64

In [45]:
df.to_csv('season_df_v1.csv', index=False)